### 12-1.  한영 번역기 만들기

In [1]:
import pandas
import tensorflow
import matplotlib
import tensorflow as tf
import os
import tarfile
import re
from tqdm import tqdm
import random
from konlpy.tag import Mecab
import numpy as np
import pandas as pd 

print(pandas.__version__)
print(tensorflow.__version__)
print(matplotlib.__version__)

1.3.3
2.6.0
3.4.3


#### Step 1. 데이터 다운로드

In [3]:
# cache_dir = "/aiffel/aiffel/assets/gd04/" 

# path_to_zip = tf.keras.utils.get_file(
#     'korean-english-park.train.tar.gz',
#     origin="https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1/korean-english-park.train.tar.gz",
#     extract=True,
#     cache_dir=cache_dir  
# )

In [4]:
file_path = '/aiffel/aiffel/assets/gd04/datasets/korean-english-park.train.tar.gz'
extract_path = '/aiffel/aiffel/assets/gd04/datasets/' 

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)  

extracted_files = os.listdir(extract_path)
print(extracted_files)

['korean-english-park.train.en', 'korean-english-park.train.ko', 'extracted', '.ipynb_checkpoints', 'korean-english-park.train.tar.gz']


In [5]:
eng_file_path = os.path.join(extract_path, 'korean-english-park.train.en')
kr_file_path = os.path.join(extract_path, 'korean-english-park.train.ko')

with open(eng_file_path, 'r', encoding='utf-8') as eng_file:
    eng_lines = eng_file.readlines()

with open(kr_file_path, 'r', encoding='utf-8') as kr_file:
    kr_lines = kr_file.readlines()

In [11]:
print(kr_lines[1])
print()
print(eng_lines[2])
print()
print(kr_lines[2])

모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.


Like all optical mice, But it also doesn't need a desk.


그러나 이것은 또한 책상도 필요로 하지 않는다.



In [12]:
eng_corpus = []
kr_corpus = []

for eng, kr in zip(eng_lines, kr_lines):
    eng_corpus.append(eng.strip()) 
    kr_corpus.append(kr.strip())  

for i in range(3):
    print(f"Eng: {eng_corpus[i]}")
    print(f"Kr: {kr_corpus[i]}")
    print()

Eng: Much of personal computing is about "can you top this?"
Kr: 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"

Eng: so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.
Kr: 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.

Eng: Like all optical mice, But it also doesn't need a desk.
Kr: 그러나 이것은 또한 책상도 필요로 하지 않는다.



< 데이터 자체가 잘못되어 있음: '모든 광마우스와 마찬가지 로' 는 'Like all optical mice,'과 매칭되는데 줄별로 자를시 매핑이 안됨 >

#### Step 2. 데이터 정제

In [13]:
def del_duplicate(eng_corpus, kr_corpus):
    corpus_set = set(zip(eng_corpus, kr_corpus)) #중복 제거
    cleaned_corpus = list(corpus_set) 
    return zip(*cleaned_corpus)  

no_duplicated_eng, no_duplicated_kr = del_duplicate(eng_corpus, kr_corpus)

In [14]:
print(no_duplicated_eng[359])
print(no_duplicated_kr[359])

“Those polls have consistently shown me much further behind than we actually are.
“그 여론조사들은 시종일관 제가 실제보다 한참 뒤쳐져 있는 것으로 보여주더군요.


In [15]:
def preprocess_sentence(text, language='ko'):
    if language == 'en':
        text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거, 소문자
        text = '<start> ' + text.lower() + ' <end>' #decoder <start>와 <end> 토큰 추가
        return text.split()
    
    elif language == 'ko':
        mecab = Mecab() 
        text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)  # 한글과 공백만 남김
        return mecab.morphs(text)  # mecab
    
preprocessed_kr = [preprocess_sentence(text, language='ko') for text in no_duplicated_kr]
preprocessed_eng = [preprocess_sentence(text, language='en') for text in no_duplicated_eng]

In [16]:
len(preprocessed_eng)

78941

In [17]:
preprocessed_kr

[['레바논',
  '북부',
  '항구',
  '도시',
  '인',
  '트리',
  '포리',
  '에서',
  '는',
  '레바논',
  '군',
  '이',
  '대포',
  '를',
  '동원',
  '해',
  '무장',
  '세력',
  '근거지',
  '를',
  '공격',
  '했',
  '다'],
 ['미',
  '국무부',
  '는',
  '이날',
  '발표',
  '한',
  '성명',
  '에서',
  '초동',
  '수사',
  '를',
  '지켜본',
  '결과',
  '이번',
  '공격',
  '목표',
  '대상',
  '은',
  '미',
  '대사관',
  '인',
  '것',
  '으로',
  '확인',
  '했',
  '다며',
  '조사',
  '가',
  '진행',
  '중',
  '이',
  '기',
  '때문',
  '에',
  '구체',
  '적',
  '인',
  '사항',
  '을',
  '밝히',
  '기',
  '는',
  '힘들',
  '다고',
  '말',
  '했',
  '다'],
 ['영국',
  '헨리',
  '왕자',
  '의',
  '홍보',
  '담당자',
  '가',
  '지난달',
  '일',
  '현지',
  '시간',
  '경찰',
  '이',
  '지난주',
  '왕실',
  '소유지',
  '에서',
  '죽',
  '은',
  '영국',
  '천연기념물',
  '인',
  '새',
  '마리',
  '가',
  '죽',
  '은',
  '사건',
  '과',
  '관련',
  '해',
  '헨리',
  '왕자',
  '를',
  '심문',
  '했',
  '다고',
  '밝혔',
  '다'],
 ['웹',
  '사이트',
  '는',
  '점점',
  '상점',
  '의',
  '홍보',
  '용',
  '도구',
  '가',
  '되',
  '어',
  '가',
  '고',
  '있',
  '습니다',
  '라고',
  '소비자',
  '소비',
  '행태',
  '를',

In [18]:
preprocessed_eng

[['<start>',
  'in',
  'nahr',
  'elbared',
  'near',
  'the',
  'northern',
  'port',
  'city',
  'of',
  'tripoli',
  'the',
  'army',
  'resumed',
  'its',
  'pounding',
  'of',
  'militant',
  'hideouts',
  'with',
  'artillery',
  'after',
  'sporadic',
  'exchanges',
  'earlier',
  'in',
  'the',
  'day',
  'sending',
  'up',
  'plumes',
  'of',
  'black',
  'smoke',
  '<end>'],
 ['<start>',
  'our',
  'conversations',
  'in',
  'yemen',
  'have',
  'led',
  'us',
  'to',
  'the',
  'conclusion',
  'that',
  'the',
  'attack',
  'was',
  'directed',
  'against',
  'our',
  'embassy',
  'according',
  'to',
  'a',
  'statement',
  'released',
  'tuesday',
  'evening',
  'since',
  'this',
  'is',
  'an',
  'ongoing',
  'investigation',
  'we',
  'are',
  'not',
  'going',
  'to',
  'talk',
  'about',
  'the',
  'specifics',
  'of',
  'the',
  'case',
  '<end>'],
 ['<start>',
  'london',
  'england',
  'cnn',
  'police',
  'questioned',
  'prince',
  'harry',
  'after',
  'two',
  

In [19]:
def filter_data(eng_corpus, kr_corpus, max_length=40):
    filtered_eng = []
    filtered_kr = []
    
    for eng, kr in zip(eng_corpus, kr_corpus):
        if len(eng) <= max_length and len(kr) <= max_length:
            filtered_eng.append(eng)
            filtered_kr.append(kr)
    
    return filtered_eng, filtered_kr

cleaned_eng, cleaned_kr = filter_data(preprocessed_eng, preprocessed_kr)

In [20]:
len(cleaned_eng)

67096

#### Step 3. 데이터 토큰화

In [21]:
top_k = 15000

In [22]:
top_k = 15000
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k, filters='')
    tokenizer.fit_on_texts(corpus)

    tokenizer.word_index = {word: index for word, index in tokenizer.word_index.items() if index < top_k}
    tokenizer.index_word = {index: word for word, index in tokenizer.word_index.items()}
    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,  maxlen=40, padding='post')

    return tensor, tokenizer

In [23]:
encoder_tensor, encoder_tokenizer = tokenize(cleaned_kr)
decoder_tensor, decoder_tokenizer = tokenize(cleaned_eng)

In [24]:
len(encoder_tokenizer.index_word)

14999

In [25]:
encoder_tensor.shape

(67096, 40)

In [34]:
encoder_tensor[128]

array([  15,  967,   11,  976,   74,  196,  109,    5,  960, 1496,    3,
        186,   85,   69, 1167,   74,   19,    2,   32,  109,  960,  844,
         11,  563,   16,  339,    4,  831, 2546,   11,  125,    8,    1,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

#### Step 4. 모델 설계

- (1) Attention Score:   
    $$e_{t,i} = v^T tanh(W_ss_t + W_hh_i)$$
   
   
- (2) Attention Score Weights:   
    $$\alpha_{t,i} = \frac{exp(e_{t,i}}{\sum_j exp(e_{t,j})}$$


- (3) Context Vector:   
    $$c_t = \sum_{i} \alpha_{t,i}h_i$$
    
    
- tf.keras.layers.Dense:
    $$W \cdot h + b$$

In [26]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units) # # W_s
        self.w_enc = tf.keras.layers.Dense(units) # # W_h
        self.w_com = tf.keras.layers.Dense(1)  # # v^T
        self.dropout = tf.keras.layers.Dropout(0.1)  

    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        #(1)
        h_enc = self.w_enc(h_enc) 
        h_dec = tf.expand_dims(h_dec, 1)  
        h_dec = self.w_dec(h_dec)
        score = self.w_com(tf.nn.tanh(h_dec + h_enc))  
        
        #(2)
        attn = tf.nn.softmax(score, axis=1)  
        attn = self.dropout(attn)  

        #(3)
        context_vec = attn * h_enc 
        context_vec = tf.reduce_sum(context_vec, axis=1) 

        return context_vec, attn

#### Encoder

- (1) Embedding Layer   
    $$e_{t} = W_ex_t$$
   
   
- (2) Hidden states:   
    $$h_t = GRU(e_t, h_{t-1})$$

#### Decoder

- (3) Input Embedding  
    $$e'_t = W'_ey_{t-1}$$
    
    
- (4) GRU
    $$s_t = GRU([e_t',c_t], s_{t-1})$$
    
- (5) Softmax
    $$\hat{y}_t = softmax(W_os_t)$$

In [27]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        #(1) (64, 30, 512)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        #(2) 
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True,
                                       dropout=0.2)
        self.dropout = tf.keras.layers.Dropout(0.2)
    
    def call(self, x):
        out = self.embedding(x)
        out = self.dropout(out)
        out = self.gru(out)
        
        return out
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        #(3)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       dropout=0.2)
        
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = self.dropout(out)
        #(4) 마지막 차원(axis=-1)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        out, h_dec = self.gru(out)
        #(5) Dense layer 차원 : (batch_size, feature_dim)
        # from_logits=True이므로 softmax 생략
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out) #vocab_size

        return out, h_dec, attn

In [28]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(encoder_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(decoder_tokenizer.index_word) + 1

units         = 512 #1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (64, 30, 512)
Decoder Output: (64, 15000)
Decoder Hidden State: (64, 512)
Attention: (64, 30, 1)


#### Step 5. 훈련

In [94]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [95]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [96]:
EPOCHS = 5

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, encoder_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)   
    for (batch, idx) in enumerate(t):
        batch_loss = train_step(encoder_tensor[idx:idx+BATCH_SIZE],
                                decoder_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                decoder_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))    
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))    

Epoch  5: 100%|██████████| 1049/1049 [03:08<00:00,  5.57it/s, Loss 3.5909]


loss를 보니 제대로 학습이 안되는것으로 확인

In [97]:
enc_train = encoder_tensor
dec_train = decoder_tensor
dec_tokenizer = decoder_tokenizer
enc_tokenizer = encoder_tokenizer

In [29]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    
    encoder_input = preprocess_sentence(sentence, language='ko')
    print(encoder_input)
    encoder_tensor = encoder_tokenizer.texts_to_sequences([encoder_input])
    print(encoder_tensor)
    inputs = tf.keras.preprocessing.sequence.pad_sequences(encoder_tensor,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    print(inputs)
    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    print(dec_hidden.shape)
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention



def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]

In [245]:
#test1 (Without dropout , without top-k, default lr (0.001))

translate("오바마는 누구야?", encoder, decoder)

['오바마', '는', '누구', '야']
[[112, 3, 1326, 619]]
[[ 112    3 1326  619    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
(1, 1024)
Input: 오바마는 누구야?
Predicted translation: the us been the us been the us been the us been the us been the us been the us been the us been the us been the us been the us been the us been the us been the 


In [81]:
#test2 (With dropout (Embedding, GRU ..) , top-k, default lr (0.001))

translate("오바마는 누구야?", encoder, decoder)

['오바마', '는', '누구', '야']
[[112, 3, 1329, 621]]
[[ 112    3 1329  621    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
(1, 512)
Input: 오바마는 누구야?
Predicted translation: obama also the president elect barack obama is the first time <end> 


In [59]:
#test3 (With dropout (Embedding, GRU ..) , top-k, default lr (0.0001))

translate("오바마는 누구야?", encoder, decoder)

['오바마', '는', '누구', '야']
[[112, 3, 1329, 621]]
[[ 112    3 1329  621    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
(1, 512)
Input: 오바마는 누구야?
Predicted translation: the first of the united states <end> 


In [98]:
#test4 (Without Embedding dropout (Include GRU dropout),  lr 0.001 )

translate("오바마는 누구야?", encoder, decoder)

['오바마', '는', '누구', '야']
[[112, 3, 1329, 621]]
[[ 112    3 1329  621    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
(1, 512)
Input: 오바마는 누구야?
Predicted translation: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 


In [30]:
# lr 0.01 > epoch4에서 loss튕김

- "the the the" 와 같이 패턴 반복 문제를 임베딩 레이어에서의 드롭아웃으로 부분적으로 해결.
- GRU단에서 드롭아웃은 단어 반복 패턴을 해결하는 데는 덜 효과적임을 확인할 수 있었는데, 반복 문제에서 앞단에서 반복 단어에 대한 의존도를 줄이는 것이 더 영향이 큰 것 같음